In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_1106335/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.discard_below_average(sort_by="r2_sup")
        self.discard_below_average(sort_by="r2_vt")
        self.discard_high_standard_deviation()
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)


In [6]:
analize = Analizer(0.9)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_18_2_9,0.999998,0.993047,0.999992,0.999970,0.999991,0.440630,1550.845648,2.004112,0.620147,1.312129,0.004600,0.663800,1.000047,0.692059,51.639099,82.110995,"Hidden Size=[3, 3], regularizer=0.2, learning_..."
1,model_18_2_10,0.999998,0.993047,0.999992,0.999973,0.999991,0.451564,1550.853228,2.136064,0.566127,1.351095,0.003726,0.671985,1.000049,0.700593,51.590076,82.061971,"Hidden Size=[3, 3], regularizer=0.2, learning_..."
2,model_18_2_11,0.999998,0.993050,0.999991,0.999977,0.999991,0.453604,1550.277852,2.241574,0.476855,1.359214,0.003331,0.673501,1.000049,0.702173,51.581063,82.052958,"Hidden Size=[3, 3], regularizer=0.2, learning_..."
3,model_18_2_12,0.999998,0.993050,0.999991,0.999982,0.999991,0.456200,1550.212215,2.351675,0.387396,1.369535,0.002610,0.675426,1.000049,0.704180,51.569647,82.041542,"Hidden Size=[3, 3], regularizer=0.2, learning_..."
4,model_18_2_13,0.999998,0.993051,0.999990,0.999984,0.999990,0.484335,1550.107325,2.581623,0.336735,1.459185,0.002883,0.695942,1.000052,0.725570,51.449956,81.921851,"Hidden Size=[3, 3], regularizer=0.2, learning_..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1591,model_36_5_4,0.998285,0.993519,0.999670,0.999805,0.999782,382.531080,1445.675376,17.027588,46.581107,31.804347,0.057726,19.558402,1.000675,20.391043,158.106380,261.710825,"Hidden Size=[21], regularizer=0.05, learning_r..."
1603,model_34_2_1,0.998204,0.995374,0.998447,0.998506,0.999115,400.666112,1031.815155,230.684102,9.965993,120.325047,0.031019,20.016646,1.000707,20.868796,158.013743,261.618188,"Hidden Size=[21], regularizer=0.2, learning_ra..."
1667,model_36_5_3,0.997905,0.993377,0.999640,0.999814,0.999784,467.207494,1477.353173,18.613599,44.436711,31.525155,0.059486,21.614983,1.000824,22.535178,157.706453,261.310898,"Hidden Size=[21], regularizer=0.05, learning_r..."
1777,model_34_2_0,0.997824,0.995548,0.998417,0.998362,0.999095,485.450469,993.103227,235.138653,10.930275,123.034464,0.028819,22.032941,1.000856,22.970928,157.629845,261.234291,"Hidden Size=[21], regularizer=0.2, learning_ra..."
